# pymysql
- 파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
- Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
- https://github.com/PyMySQL/PyMySQL/
- https://pymysql.readthedocs.io/en/latest/

In [3]:
!pip install pymysql

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 44.8/44.8 kB ? eta 0:00:00


# 설치
- 조건
    - python version 3.6 이상
    - mysql version 5.6 이상
- 설치
    - `pip install PyMySQL`
    - `conda install -c conda-forge pymysql`

# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host="DBMS 서버 ip", 
                                     port="port번호", 
                                     user="계정명", 
                                     password="비밀번호", 
                                     db="연결할데이터베이스이름", 
                                     charset='utf8')
    ```
    - port 번호 기본값: 3306
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
3. Cusror를 이용해 SQL문 전송
    ```python
        cursor.execute("sql문")
    ```
4. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다. 
    ```python
    cursor.close()
    connection.close()
    ```

# 예제

## 테이블 생성

In [4]:
create_sql = """
create table member(
  id  int  primary key  auto_increment, 
  name varchar(30) not null,
  email varchar(100) not null unique,
  tall  decimal(5, 2),
  birthday  date, 
  created_at datetime
)
"""

In [5]:
import pymysql
# 1. DB와 연결
# create database testdb;
conn = pymysql.connect(host="127.0.0.1", # DBMS의 ip주소
                       port=3306,        # port 번호
                       user='playdata',  # username
                       password='1111',  # password
                       db='testdb'       # 연결할 database(schema) 이름
                      )
print(type(conn))

<class 'pymysql.connections.Connection'>


In [6]:
# 2. Connection 으로부터 Curosr객체를 생성
## Cursor: SQL문을 DB에 전송 그 결과를 받아주는 객체. -> sql 전송 메소드를 제공.
cursor = conn.cursor()
print(type(cursor))

<class 'pymysql.cursors.Cursor'>


In [7]:
# 3. sql 전송.
cursor.execute(create_sql) #return -> sql이 적용된 데이터의 개수.

0

In [8]:
# 4. 연결 닫기 -> cursor, connection
cursor.close()
conn.close()

## DML
### insert

In [11]:
sql = "insert into member (name, email, tall, birthday, created_at) \
values ('홍길동', 'a@a.com', 175.22, '2000-10-10', now())"

# pymysql: manual commit
with pymysql.connect(host="127.0.0.1", port=3306, 
                     user='playdata', password='1111', db='testdb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(sql)
        print("insert된 행수:", cnt)
        conn.commit() # insert, delete, update 처리후 commit() 을 해야 적용된다.
 

insert된 행수: 1


In [13]:
sql1 = "insert into member (name, email, tall, birthday, created_at) \
values ('홍길동', 'a4@a.com', 175.22, '2000-10-10', now())"
sql2 = "insert into member (name, email, tall, birthday, created_at) \
values ('이순신', 'a2@a.com', 175.22, '2000-10-10', now())"
sql3 = "insert into member (name, email, tall, birthday, created_at) \
values ('유관순', 'a3@a.com', 175.22, '2000-10-10', now())"
with pymysql.connect(host="127.0.0.1", port=3306, 
                     user='playdata', password='1111', db='testdb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(sql1)
        print("insert된 행수:", cnt)
        cnt = cursor.execute(sql2)
        print("insert된 행수:", cnt)
        cnt = cursor.execute(sql3)
        print("insert된 행수:", cnt)
        conn.commit() # insert, delete, update 처리후 commit() 을 해야 적용된다.

insert된 행수: 1
insert된 행수: 1
insert된 행수: 1


### Parameterized Query
- Parameterized Query
    - SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 tuple로 제공한다.
    - query문을 쉽게 작성할 수 있는 장점이 있다.        

In [15]:
sql_template = "insert into member (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, now())"
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='testdb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.execute(sql_template, ('유재석', 'b@a.com', 182.54, '2000-10-10'))
        print(cnt)
        conn.commit()

1


### Parameterized Query를 이용해 여러개 행 insert

#### for문 사용

In [18]:
from datetime import date, datetime
date(2000, 2, 3)
datetime(2000, 2, 3, 10, 20, 30)
datetime.now()

datetime.datetime(2023, 9, 8, 17, 29, 35, 365845)

In [19]:
from datetime import date, datetime
# python: datetime.date - sql: date
# datetime.time - sql: time
# datetime.datetime - sql: datetime, timestamp
datas = [
    ['name1', 'abc1@abc.com', 165, date(2000,1,12)],
    ['name2', 'def1@abc.com', 175, date(1995,12,20)],
    ['name3', 'ghi1@abc.com', 185, date(1988, 7, 21)]
]

sql_template = "insert into member (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, now())"
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='testdb') as conn:
    with conn.cursor() as cursor:
        cnt = 0
        for data in datas:
            i = cursor.execute(sql_template, data) #placeholder에 넣을 값들은 tuple, list에 넣어 제공.
            cnt += i
        print("inser된 행수:", cnt)
        conn.commit()

inser된 행수: 3


#### executemany() 사용
- insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.

In [23]:
datas = [
    ['name1', 'abc3@abc.com', 165, date(2000,1,12)],
    ['name2', 'def3@abc.com', 175, date(1995,12,20)],
    ['name3', 'ghi3@abc.com', 185, date(1988, 7, 21)]
]

sql_template = "insert into member (name, email, tall, birthday, created_at) \
values (%s, %s, %s, %s, now())"
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='testdb') as conn:
    with conn.cursor() as cursor:
        try:
            cnt = cursor.executemany(sql_template, datas)
            print("insert 개수: ", cnt)
            conn.commit()
        except Exception as e:
            print("예외발생: ", e)
            conn.rollback()

insert 개수:  3


### update/delete
- 코딩 절차는 insert 와 동일

In [ ]:
update_sql = "update member set tall=tall+10 where id = %s"

In [ ]:
delete_sql = "delete from member where id = %s"

## select (DQL - Data Query Language)
- 조회결과 조회
    - cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
- fetch메소드
    - **fetchall()**
        - 조회한 모든 행을을 반환
    - **fetchmany(size=개수)**
        - 지정한 size개수 만큼 반환
    - **fetchone()**
        - 조회결과 중 첫번째 행만 반환
        - 주로 pk 동등 조건으로 조회한 경우 사용

### fetchall()

### fetchone()

### fetchmany()

### cursor 는 iterable 타입
- for문에 select 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.

# TODO

다음 함수들을 구현하시오.   
각 함수들은 member 테이블과 관련해서 CRUD를 처리하는 함수들 입니다.

1. name, email, tall, birthday를 매개변수로 받아서 insert하는 함수. (id는 자동증가, created_at은 실행시점의 일시가 insert되도록 한다.)
2. id, name, email, tall, birthday를 매개변수로 받아서 id의 member의 나머지 정보를 update하는 함수. (created_at은 update하지 않는다.)
3. id를 매개변수로 받아서 그 member 를 삭제하는 함수.
4. 이름을 매개변수로 받아서 그 이름의 member를 삭제하는 함수.
5. id를 매개변수로 받아서 그 id의 회원 정보를 조회하여 반환하는 함수.
6. name을 매개변수로 받아서 그 이름이 들어간 회원의 정보를 조회하여 반환하는 함수.
7. birthday를 매개변수로 받아서 그 생일의 회원의 정보를 조회하여 반환하는 함수.
8. tall 값을 두개 받아서 그 범위의 tall인 회원들의 정보를 조회하여 반환하는 함수.


- insert, update, delete 는 적용된 행의 개수를 반환한다.
- select 처리 함수는 조회결과를 반환한다.


# TODO
- hr database의 emp 테이블의 데이터를 다루는 함수들을 구현

1. 직원 정보들(emp의 컬럼들)을 매개변수로 받아 insert 하는 함수

2. 수정할 직원 정보들을 매개변수로 받아 받은 emp_id의 직원의 나머지 컬럼값들을 수정하는 함수

3. emp_id를 매개변수로 받아 그 직원을 삭제하는 함수

4. job을 매개변수로 받아 그 업무를 하는 직원들을 삭제하는 함수

5. 전체 직원정보를 모두 조회해서 반환하는 함수

6. emp_id를 매개변수로 받아 그 직원 정보를 조회해서 반환하는 함수

7. dept_name을 매개변수로 받아서 그 부서에 소속된 직원들의 정보를 조회해서 반환하는 함수

8. salary 범위를 매개변수로 받아 그 범위의 salary를 받는 직원들의 정보를 조회해서 반환하는 함수

9. emp_id를 매개변수로 받아서 그 직원의 커미션이 얼마인지를(salary * comm_pct) 조회해 반환하는 함수

10. dept_name을 매개변수로 받아서 부서별 급여 통계정보(합계, 최대, 최소, 평균, 표준편차)를 반환하는 함수